In [1]:
from gevent import monkey
monkey.patch_all(thread=False)

from gevent.pool import Pool
pool = Pool(30)

from gevent.lock import BoundedSemaphore
semaphore_correct, semaphore_error, semaphore_remain = BoundedSemaphore(1), BoundedSemaphore(1), BoundedSemaphore(1)

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import os
import shutil

directory_out = './file/3.spec'

if os.path.exists(directory_out):
    shutil.rmtree(directory_out)
os.makedirs(directory_out)

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

from tqdm import tqdm
import re

# = = = = = = = = = = = = = = =

directory_in = './file/2.oem'

list_file = [file for file in sorted(os.listdir(directory_in)) if file not in ['.DS_Store'] and os.path.isfile(f'{directory_in}/{file}')]
for file in list_file:

    # = = = = = = = = = = = = = = =

    list_dict_correct, list_series_error, df_input = [], [], pd.read_excel(f'{directory_in}/{file}',
                                                                           header=0,
                                                                           dtype=str).fillna('')

    count_remain = len(df_input)

    print(f'总数量：{count_remain}')
    print()

    # = = = = = = = = = = = = = = =

    def crawler(crawler_tuple):
        global list_dict_correct, list_series_error, count_remain

        crawler_status, (crawler_index, crawler_series) = False, crawler_tuple

        # = = = = = = = = = = = = = = =

        try:
            count_retry = 0
            while True:
                count_retry += 1

                try:
                    url_check = crawler_series['[UAC] Url']

                    resp = requests.get(f'https://uacparts.com/Catalog/Specs?part={crawler_series['[UAC] Part Number']}',
                                        headers=get_header(),
                                        proxies=get_proxy(),
                                        timeout=(5, 15))

                    if resp.status_code == 200:
                        soup = BeautifulSoup(resp.text, 'lxml')
                        html = etree.HTML(str(soup))

                        if html.xpath('//form[@action="/Catalog/Specs"]'):
                            break
                except KeyboardInterrupt:
                    break
                except:
                    continue

            # = = = = = = = = = = = = = = =

            list_tr, dict_count = html.xpath('//tr[@class="awe-row"]'), {}
            for tr in list_tr:
                key = tr.xpath('./td/text()')[0].strip()
                if key in dict_count:
                    dict_count[key] += 1
                else:
                    dict_count[key] = 1

            dict_spec, dict_no = {}, {}
            for i, tr in enumerate(list_tr):
                key = tr.xpath('./td/text()')[0].strip()
                if dict_count[key] == 1:
                    dict_spec[str(i)] = {key: tr.xpath('./td/text()')[1].strip()}
                else:
                    if key in dict_no:
                        dict_no[key] += 1
                    else:
                        dict_no[key] = 1
                    dict_spec[str(i)] = {f'{key}-{dict_no[key]}': tr.xpath('./td/text()')[1].strip()}

            # = = = = = = = = = = = = = = =

            with semaphore_correct:
                list_dict_correct.append({'Part Number': crawler_series['Part Number'],
                                          '[UAC] Part Number': crawler_series['[UAC] Part Number'],
                                          '[UAC] OEM': crawler_series['[UAC] OEM'],
                                          '[UAC] Picture': crawler_series['[UAC] Picture'],
                                          '[UAC] Url': crawler_series['[UAC] Url'],
                                          '[UAC] Json_Src': crawler_series['[UAC] Json_Src'],
                                          '[UAC] Json_Note': crawler_series['[UAC] Json_Note'],
                                          '[UAC] Json_Spec': json.dumps(dict_spec)})

            # = = = = = = = = = = = = = = =

            crawler_status = True
        except:
            crawler_series['Check_Url'] = url_check

            with semaphore_error:
                list_series_error.append(crawler_series)

        # = = = = = = = = = = = = = = =

        if list_file.index(file) % 3 == 0 and list_file.index(file) != 0 and count_remain == 1 or count_remain % 300 == 0 and count_remain != 0:
            clear_output()
        print(f'''[{crawler_status}] - [请求{count_retry}次] - [剩余{count_remain - 1}条] - [{file.removesuffix('.xlsx')}（{(list_file.index(file) + 1) / len(list_file) * 100:.2f}%）] - [{datetime.now().strftime('%Y/%m/%d %H:%M:%S')}] - {crawler_series['[UAC] Part Number']}. {url_check}\n''')

        with semaphore_remain:
            count_remain -= 1

    # = = = = = = = = = = = = = = =

    pool.map(crawler, df_input.iterrows())

    print('输出ing...')
    print()
    if list_dict_correct:
        df_correct = pd.DataFrame(list_dict_correct).sort_values(by=['Part Number'],
                                                                 ascending=[True],
                                                                 ignore_index=True)
        df_correct.to_excel(f'''{directory_out}/{file.removesuffix('.xlsx')}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
    if list_series_error:
        df_error = pd.DataFrame(list_series_error).sort_values(by=['Part Number'],
                                                               ascending=[True],
                                                               ignore_index=True)
        df_error.to_excel(f'''./file/{file.removesuffix('.xlsx')}-spec_error.xlsx''', index=False)
        print('爬虫存在error')
        print()

# = = = = = = = = = = = = = = =

list_file = [file for file in sorted(os.listdir(directory_out)) if file not in ['.DS_Store'] and os.path.isfile(f'{directory_out}/{file}')]
for file in tqdm(list_file, desc='Progress', ncols=77):
    os.rename(f'{directory_out}/{file}',
              f'''{directory_out}/{re.sub(r'-[0-9]{8}_[0-9]{6}.xlsx$', '.xlsx', file)}''')

print('Done ~')

总数量：10

[True] - [请求1次] - [剩余9条] - [crawler_1（50.00%）] - [2024/10/22 09:43:46] - HA 10029C. https://uacparts.com/Catalog/SearchIndex?search=HA10029C

[True] - [请求1次] - [剩余8条] - [crawler_1（50.00%）] - [2024/10/22 09:43:46] - HA 10040C. https://uacparts.com/Catalog/SearchIndex?search=HA10040C

[True] - [请求1次] - [剩余7条] - [crawler_1（50.00%）] - [2024/10/22 09:43:46] - HA 10066C. https://uacparts.com/Catalog/SearchIndex?search=HA10066C

[True] - [请求1次] - [剩余6条] - [crawler_1（50.00%）] - [2024/10/22 09:43:47] - HA 10018C. https://uacparts.com/Catalog/SearchIndex?search=HA10018C

[True] - [请求1次] - [剩余5条] - [crawler_1（50.00%）] - [2024/10/22 09:43:47] - HA 10038C. https://uacparts.com/Catalog/SearchIndex?search=HA10038C

[True] - [请求1次] - [剩余4条] - [crawler_1（50.00%）] - [2024/10/22 09:43:47] - HA 10068C. https://uacparts.com/Catalog/SearchIndex?search=HA10068C

[True] - [请求1次] - [剩余3条] - [crawler_1（50.00%）] - [2024/10/22 09:43:47] - HA 10020C. https://uacparts.com/Catalog/SearchIndex?search=HA10020C

Progress: 100%|██████████████████████████████| 2/2 [00:00<00:00, 4082.05it/s]

Done ~
